# Broadcast

Nous allons voir ici un concept utiliser avec Spark Core et utilisé sous une autre forme avec Spark SQL. Il s'agit des variables _broadcast_.

Nous sommes dans une situation où nous avons récupéré une dataset et nous voulons croiser ses données avec celle d'un référentiel. Par exemple, nous avons récupéré des données de commandes client. Sauf que dans le champ client, nous n'avons que des identifiants et pas l'identité du client. Cette relation entre identité et identifiant du client est fourni dans un autre fichier. On pourrait charger ce fichier dans un RDD, mais il est possible que ce fichier des identifiants soit suffisamment petit pour tenir en mémoire et qu'il peut s'avérer plus intéressant de copier ses données dans chaque exécuteur.

C'est dans ce cas que les variables _broadcast_ peuvent être utilisées.

## Prélude

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
    .appName("Spark tuning - broadcast")
    .master("local[*]")
    .config("spark.ui.showConsoleProgress", "True")
    .config("spark.executor.memory", "2g")
    .getOrCreate())

# Access the JVM and import the required Java classes
jvm = spark.sparkContext._jvm
Level = jvm.org.apache.logging.log4j.Level
Configurator = jvm.org.apache.logging.log4j.core.config.Configurator

# Set the root level to OFF
Configurator.setRootLevel(Level.OFF)

## Chargement d'un fichier de mapping

Dans le code ci-dessous, nous allons charger un fichier contenant des correspondances entre identifiant client et nom de client.

In [ ]:
mapping_filename = "data/client-mapping.csv"

mapping = {}
with open(mapping_filename, 'r') as file:
    # Skip header
    next(file)
    
    for line in file:
        fields = line.strip().split(",")
        mapping[fields[0]] = fields[1]

print(mapping)

## Chargement des commandes clients

In [ ]:
from pyspark.sql import Row
from datetime import datetime

rawData = spark.sparkContext.textFile("data/orders.csv", 4)
header = rawData.first()
data = rawData.filter(lambda line: line != header)

def Order(id, clientId, timestamp, product, price):
    return Row(id=id, clientId=clientId, timestamp=timestamp, product=product, price=price)

def to_local_date_time(s: str) -> datetime:
    return datetime.strptime(s, '%Y-%m-%dT%H:%M:%S')

def line_to_order(line: str) -> Row:
    fields = line.split(",")
    return Order(
        id=fields[0],
        clientId=fields[1],
        timestamp=to_local_date_time(fields[2]),
        product=fields[3],
        price=float(fields[4])
    )

# Assuming data is an RDD
orders = data.map(line_to_order)

spark.createDataFrame(orders).show(10)

## Broadcast

Une variable _broadcast_ se crée à partir du SparkContext, en utilisant la méthode `.broadcast()` en passant en paramètre la valeur à diffuser sur les exécuteurs.

In [ ]:
broadcast_mapping = spark.sparkContext.broadcast(mapping)

La récupération de la valeur associée à une variable _broadcast_ se fait en appelant la méthode `.value`.

Comme notre table de correspondance `broadcastMapping` représente une collection de type `Map` et que nous souhaitons récupérer le nom d'un client par rapport à son identifiant (`order.clientId`), s'il est présent, nous allons utiliser la méthode `.getOrElse(key, default)` pour récupérer le nom du client.

In [ ]:
def map_order(order):
  order_dict = order.asDict()
  order_dict['clientId'] = ???
  return Row(**order_dict)

mappedOrders = orders.map(map_order)

Nous obtenons maintenant cet affichage :

In [ ]:
spark.createDataFrame(mappedOrders).show(10)